In [1]:
import cv2.aruco as aruco
import pickle
import time

In [2]:
import sys
import torch
sys.path.insert(0,r"D:\git\AdversarialRendering\utils")
import tqdm
from classfier import *
orig_clases = torch.tensor([817, 705, 609, 586, 436, 627, 468, 621, 803, 407, 408, 751, 717,866])

import torchvision
tt = torchvision.transforms.ToTensor()

In [3]:
# load matrix and distortion from file
cameraMatrix, dist = pickle.load(open("calibration.pkl", "rb"))

In [4]:
import cv2
import matplotlib.pyplot as plt

camera_index = 0
cap = cv2.VideoCapture(camera_index, cv2.CAP_DSHOW) # this is the magic!


In [5]:
adv_patch = cv2.imread(r"D:\git\AdversarialRendering\patches\13_05_pred.png")


In [6]:
USE_aruco = False


In [7]:
import cv2
import cv2.aruco as aruco
import numpy as np

# Define parameters for ArUco marker detection
aruco_dict_type = cv2.aruco.DICT_6X6_250 # Change dictionary type if needed
marker_length = 0.05  # Marker length in meters (adjust as needed)
aruco_dict = cv2.aruco.getPredefinedDictionary(aruco_dict_type)

marker_id = 41
marker_size = 500  # Size in pixels
# marker_image = cv2.aruco.generateImageMarker(aruco_dict, marker_id, marker_size)

import time
# add white border to marker image
cv2.destroyAllWindows()

border_size = 2

proj_marker_image = cv2.aruco.generateImageMarker(aruco_dict, marker_id, marker_size)

proj_marker_image = cv2.copyMakeBorder(proj_marker_image, border_size, border_size, border_size, border_size, cv2.BORDER_CONSTANT, value=(255, 255, 255))

proj_marker_image = cv2.cvtColor(proj_marker_image, cv2.COLOR_BGR2RGB)

In [8]:
orig_aruco_corners = np.array([[0, 0], [proj_marker_image.shape[1], 0], [proj_marker_image.shape[1], proj_marker_image.shape[0]], [0, proj_marker_image.shape[0]]], dtype=np.float32)


In [9]:
aruco_dict = aruco.getPredefinedDictionary(aruco_dict_type)
parameters = aruco.DetectorParameters()

# Detect ArUco markers
detector = aruco.ArucoDetector(aruco_dict, parameters)

In [10]:
for i in range(10):
    ret, frame = cap.read()
    if not ret:
        print("Failed to capture image")
        break
orig_frame = frame.copy()


In [11]:

# import required libraries
import cv2
import numpy as np
drawing = False

done = False
ix,iy = -1,-1

rect_corners = None

# define mouse callback function to draw circle
def draw_rectangle(event, x, y, flags, param):
   global ix, iy, drawing, img, done, rect_corners
   if event == cv2.EVENT_LBUTTONDOWN:
      drawing = True
      ix = x
      iy = y

   elif event == cv2.EVENT_LBUTTONUP:
      drawing = False
      cv2.rectangle(img, (ix, iy),(x, y),(0, 255, 255),-1)
      rect_corners = [(ix, iy), (x, y)]
      done = True





# Create a black image
img = np.zeros((1080,1920,3), np.uint8)

# Create a window and bind the function to window
cv2.namedWindow("Rectangle Window", cv2.WND_PROP_FULLSCREEN)

# Connect the mouse button to our callback function
cv2.setMouseCallback("Rectangle Window", draw_rectangle)

# display the window
while True:
   cv2.imshow("Rectangle Window", img)
   if cv2.waitKey(10) == 27:
    break
# if done:
#     time.sleep(5)
    
# cv2.destroyAllWindows()


# img = img[:,:,-1]
orig_img_mask = img.copy()

non_zero_indices = np.nonzero(img[:,:,-1])

a,b,c,d = non_zero_indices[0].min(), non_zero_indices[0].max(), non_zero_indices[1].min(), non_zero_indices[1].max()
width = d - c
height = b - a


to_place = cv2.resize(proj_marker_image, (width+1, height+1), interpolation=cv2.INTER_AREA)


img[img[:,:,-1]!=0] = to_place.reshape(( (width+1) * (height+1), 3))

cv2.imshow("Rectangle Window", img)

cv2.waitKey(1)


orig_rect_corners = [(rect_corners[0][0], rect_corners[0][1]), (rect_corners[1][0], rect_corners[0][1]), (rect_corners[1][0], rect_corners[1][1]), (rect_corners[0][0], rect_corners[1][1])]

ids = []
orig_proj_corners = np.array(orig_rect_corners)
while ids is None or 41 not in ids or (USE_aruco and 42 not in ids):
    for i in range(10):
        ret, frame = cap.read()
        if not ret:
            print("Failed to capture image")
            break

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    corners, ids, _ = detector.detectMarkers(gray)

print(ids)

corners_img_proj = corners[np.where(ids == 41)[0].item()]

H_proj, _ = cv2.findHomography(orig_proj_corners, corners_img_proj)

H_proj_inv = np.linalg.inv(H_proj)


if USE_aruco:
    corners_img_aruco = corners[np.where(ids == 42)[0].item()]
    H_aruco, _ = cv2.findHomography(orig_aruco_corners, corners_img_aruco)
    inv_orig_aruco = np.linalg.inv(H_aruco)


to_place = cv2.resize(adv_patch, (width+1, height+1), interpolation=cv2.INTER_AREA)

img_patch = orig_img_mask.copy()

img_patch[orig_img_mask[:,:,-1]!=0] = to_place.reshape(( (width+1) * (height+1), 3))

cv2.imshow("Rectangle Window", img_patch)

cv2.waitKey(1)



[[41]]


-1

In [12]:
import cv2    
freak = cv2.xfeatures2d.FREAK_create()
fast = cv2.FastFeatureDetector_create()

In [13]:
sift = cv2.SIFT_create()

def find_M(img1, img2):

    img1 = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)
    img2 = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY)
    # find the keypoints and descriptors with SIFT
    kp1 = fast.detect(img1, None)
    kp2 = fast.detect(img2, None)
    kp1, des1 = freak.compute(img1, kp1)
    kp2, des2 = freak.compute(img2, kp2)

    # kp1, des1 = freak.detectAndCompute(img1,None)
    # kp2, des2 = freak.detectAndCompute(img2,None)

    FLANN_INDEX_KDTREE = 1
    index_params = dict(algorithm = FLANN_INDEX_KDTREE, trees = 5)
    search_params = dict(checks = 50)

    # flann = cv2.FlannBasedMatcher(index_params, search_params)
    flann = cv2.BFMatcher()

    matches = flann.knnMatch(des1,des2,k=2)

    good = []
    for m,n in matches:
        if m.distance < 0.7*n.distance:
            good.append(m)

    src_pts = np.float32([ kp1[m.queryIdx].pt for m in good ]).reshape(-1,1,2)
    dst_pts = np.float32([ kp2[m.trainIdx].pt for m in good ]).reshape(-1,1,2)

    M, mask = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC,5.0)    

    return M

In [14]:
cap_frames =[]
while True:
    # Capture frame-by-frame
    ret, frame = cap.read()
    if not ret:
        print("Failed to capture image")
        break
    cap_frames.append(frame.copy())

    M = find_M(orig_frame,frame)


    
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    corners, ids, _ = detector.detectMarkers(gray)
    # print(ids)

    if not USE_aruco or ids is not None and 42 in ids:

        if USE_aruco:
            H_aruco, _ = cv2.findHomography(orig_aruco_corners, corners[np.where(ids == 42)[0].item()][0])
            
            cur_corners_img_aruco = corners[np.where(ids == 42)[0].item()]

            diff_H =    H_aruco @ inv_orig_aruco

        warped_proj_orig_aruco = cv2.warpPerspective(img, H_proj , (frame.shape[1], frame.shape[0])) 

        warped_proj_orig = cv2.warpPerspective(img_patch, H_proj , (frame.shape[1], frame.shape[0])) 

        warped_proj = cv2.warpPerspective(warped_proj_orig, M  , (frame.shape[1], frame.shape[0]))

        warped_proj_aruco = cv2.warpPerspective(warped_proj_orig_aruco, M  , (frame.shape[1], frame.shape[0]))

        warped_proj_inv = cv2.warpPerspective(warped_proj, H_proj_inv, (img.shape[1], img.shape[0]))
        warped_proj_inv_aruco = cv2.warpPerspective(warped_proj_aruco, H_proj_inv, (img.shape[1], img.shape[0]))

        
        cv2.imshow("Rectangle Window", warped_proj_inv)

        cv2.imshow("warped_proj", warped_proj)


        key = cv2.waitKey(1)
        if key == ord('q'):
            break

        pred = resnet_predict(tt(frame).cuda().unsqueeze(0))
        cv2.putText(frame, str(pred), (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2, cv2.LINE_AA)

        cv2.imshow("frame", frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    
    